In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import numpy as np

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

# Building the CNN for Brain MRI Classification

In [36]:
# Scaling our data between 0-1
X = X/255.0
y = np.array(y) # MUST CONVERT y TO AN ARRAY OR ELSE WE GET ERRORS WHEN FITTING
KERNEL_SIZE_ = 20
POOL_SIZE_ = 5

model = Sequential()

# First layer
model.add(Conv2D(70, 
          kernel_size=(KERNEL_SIZE_, KERNEL_SIZE_), # Size of our kernel during Convolution
          input_shape=X.shape[1:]) # The shape of X should return (3000, 220, 250, 1) (But we don't need the 3000, which is how many feature sets we have)
         )
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(POOL_SIZE_,POOL_SIZE_)))

# Second layer
model.add(Conv2D(70, kernel_size=(KERNEL_SIZE_, KERNEL_SIZE_)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(POOL_SIZE_, POOL_SIZE_)))

'''
# Third layer
model.add(Conv2D(70, kernel_size=(KERNEL_SIZE_, KERNEL_SIZE_)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(POOL_SIZE_, POOL_SIZE_)))

# Fourth layer
model.add(Conv2D(70, kernel_size=(KERNEL_SIZE_, KERNEL_SIZE_)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(POOL_SIZE_, POOL_SIZE_)))
'''

# Third Layer - Flatten the 2D image to make it fit into our 1D dense layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

# Fourth Layer - Output layer
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Out of our entire training data, part of it (Which is 10% in this case) is our validation
model.fit(X, y, batch_size=40, validation_split=0.1, epochs=3)


Epoch 1/3
68/68 [==============================] - 226s 3s/step - loss: 0.6640 - accuracy: 0.6044 - val_loss: 0.6054 - val_accuracy: 0.6833
Epoch 2/3
68/68 [==============================] - 229s 3s/step - loss: 0.5559 - accuracy: 0.7100 - val_loss: 0.5505 - val_accuracy: 0.6933
Epoch 3/3
68/68 [==============================] - 237s 3s/step - loss: 0.5105 - accuracy: 0.7526 - val_loss: 0.5104 - val_accuracy: 0.7800


# Saving Model

In [38]:
model.save("MRI_classification_CNN.h5")

In [39]:
later_model = load_model("MRI_classification_CNN.h5")